# This script is for BERT (or DeBERTa training) Demonstration.

In [1]:
import os
import pandas as pd
# from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
# sklearn.metrics.accuracy_score
# import xgboost as xgb
import numpy as np
import itertools 
import tqdm
from time import sleep
import random
# from docx import Document
import docx2txt

In [2]:
import torch

from transformers import BertTokenizer, BertForSequenceClassification
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader
from accelerate.utils.memory import clear_device_cache
from torch.optim import AdamW
from transformers import get_scheduler
from accelerate.test_utils.testing import get_backend
from tqdm.auto import tqdm
import evaluate

In [3]:
import warnings 
warnings.filterwarnings("ignore")

In [4]:
# This is just for disabling warning. If you are debugging bypass this cell
from transformers import logging

# logging.set_verbosity_warning()
logging.set_verbosity_error() 

In [5]:
from datasets import Dataset
# Prepare the model input features
def tokenize(batch):
    return tokenizer(batch["summarization"], padding=True, truncation=True)

In [6]:
# Please specify your model here
device, _, _ = get_backend()
model_name = "bert-base-uncased"
# model_name = "/data/project/deberta-v3-base"
if "bert" in model_name.split("-"):
    tokenizer = BertTokenizer.from_pretrained(model_name)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
# mannually generate the search grid
def makeGrid(pars_dict):  
    keys=pars_dict.keys()
    combinations=itertools.product(*pars_dict.values())
    ds=[dict(zip(keys,cc)) for cc in combinations]
    return ds

## Data Preparation

In [8]:
df_raw = pd.read_excel("../Dataset/MergedDataset_231207_ForElly_Excel.xlsx")

# Remove "." in the feature column
row_num, column_num = df_raw.shape
for rowID in range(row_num):
    for columnID in range(column_num):
        if "." == df_raw.iloc[rowID, columnID]:
            df_raw.iloc[rowID, columnID] = np.nan

# prepare the feature list
featureList = []
for item in df_raw.columns:
    if item.startswith("Item"):
        featureList.append(item)

patientList = []
roundList = []
for idx, row in df_raw.iterrows():
    patient = int(str(row["IAPTus_Num"]).split("_")[0])
    if "_" in str(row["IAPTus_Num"]):
        Round = int(str(row["IAPTus_Num"]).split("_")[1])
    else:
        Round = 0
    patientList.append(patient)
    roundList.append(Round)
df_raw["patient"] = patientList
df_raw["round"] = roundList

In [9]:
df_referral = pd.read_json("../Dataset/ReferralLetterSummary.json", lines=True)

In [10]:
df = pd.merge(df_raw, df_referral, on=["patient","round"])

## Label Preparation

### Here I use RecoveryDesc, not the EndDesc as the prediction target. EndDesc has 4 label categories and Referral Letter has only 44 data points, so it does not meet the conditions for tuning.

In [11]:
df = df[df["RecoveryDesc"] == df["RecoveryDesc"]]
df = df[(df["RecoveryDesc"].isin(["At recovery", "Not at recovery"]))]
df.reset_index().drop("index", axis=1)
print(df.shape)

labelList = []
for idx, row in df.iterrows():
    if (row["ReliableChangeDesc"] == "Reliable improvement") & (row["ReliableRecoveryDesc"] == "Reliable recovery") & (row["RecoveryDesc"] == "At recovery"):
        labelList.append(1)
    else:
        labelList.append(0)
df["labels"] = np.array(labelList, dtype=float)
labelList = ["labels"]

(43, 285)


In [13]:
df = df[["IAPTus_Num_x","patient","round","text", "summarization", "labels"]]
df = df.reset_index().drop("index", axis=1)

In [1]:
df

NameError: name 'df' is not defined

# Training Code 

In [22]:
def train_predict(df, param, model_name):
    # KFold Split
    skf = StratifiedKFold(n_splits=5)
    print(skf)
    FoldList = []
    for i, (train_index, val_index) in enumerate(skf.split(df["summarization"], df["labels"])):
        train = df.iloc[train_index]
        test = df.iloc[val_index]
        
        train = train[["summarization", "labels"]]
        trainDataset = Dataset.from_pandas(train, preserve_index=False).map(tokenize, batched=True, batch_size=None)
        trainDataset.set_format("torch")

        test = test[["summarization", "labels"]]
        testDataset = Dataset.from_pandas(test, preserve_index=False).map(tokenize, batched=True, batch_size=None)
        testDataset.set_format("torch")

        # Now I only have 16G GPU Memory, so for BERT the biggest batch size I can reach is 32, while for DeBERTa it is 4. 
        # We'd better find a machine with bigger GPU memory to run DeBERTa with batch size 32
        if "bert" in model_name.split("-"):
            train_dataloader = DataLoader(trainDataset, shuffle=True, batch_size=32)
        else:
            # train_dataloader = DataLoader(trainDataset, shuffle=True, batch_size=32)
            train_dataloader = DataLoader(trainDataset, shuffle=True, batch_size=4)
        eval_dataloader = DataLoader(testDataset, batch_size=1)
        aucList = []
        for _ in range(5):
            aucList.append(singleTrain(train_dataloader, eval_dataloader, param, model_name))
        
        FoldList.append(aucList)
    return FoldList     

In [23]:
def singleTrain(train_dataloader, eval_dataloader, param, model_name):
    # Clean GPU 
    with torch.no_grad():
        torch.cuda.empty_cache()

    if "bert" in model_name.split("-"):
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
    optimizer = AdamW(model.parameters(), lr=param["lr"])        
    num_epochs = param["epoch"]
    num_training_steps = num_epochs * len(train_dataloader)
    model.to(device)    
    model.train()
    for epoch in range(num_epochs):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
    
            optimizer.step()
            optimizer.zero_grad()
            # progress_bar.update(1)      

    finalResult = []
    finalLabel = []
    model.eval()    
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
        with torch.no_grad():
            outputs = model(**batch)
    
        logits = outputs.logits
        predictions = torch.sigmoid(logits)
        # print(predictions)
        finalResult += predictions.tolist()
        finalLabel += batch["labels"].tolist()
        # metric.add_batch(predictions=predictions, references=batch["labels"])
    # print(finalResult)
    # print(finalLabel)
    # print(roc_auc_score(finalLabel, finalResult))
    auc = roc_auc_score(finalLabel, finalResult)  
    return auc

## Test for the training code

In [24]:
param_dict = {
    "lr": 1e-6,
    "epoch": 3
}

In [25]:
results = train_predict(df, param_dict, model_name)

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [26]:
results

[[0.7, 0.65, 0.45, 0.30000000000000004, 0.75],
 [0.5, 0.6499999999999999, 0.19999999999999996, 0.35, 0.7],
 [0.45, 0.04999999999999999, 0.35, 0.5499999999999999, 0.44999999999999996],
 [0.125, 0.625, 0.5625, 0.5, 0.3125],
 [0.5, 0.6875, 0.625, 0.625, 0.5625]]

In [27]:
np.mean(results)

0.489

# Grid Search for Hyperparameter settings

In [16]:
def MyGridSearch(df, param_dict):
    searchSpace = makeGrid(param_dict)
    print("Search Space Size:" + len(searchSpace).__str__())
    resultList = []
    for param in tqdm(searchSpace):
        #sleep(1)
        scores = train_predict(df, param)
        result = {}
        result["auc_mean"] = np.mean(scores)
        result["auc_std"] = np.std(scores)
        result["auc"] = scores
        result["param"] = param
        resultList.append(result)
    sortedResult = sorted(resultList, key=lambda x: x["auc_mean"], reverse=True)
    print(sortedResult[:5])
    return sortedResult

In [17]:
param_dict = {
    "lr": [1e-8, 5e-8, 1e-7, 5e-7, 8e-7, 1e-6, 2e-6, 5e-6, 1e-5],
    "epoch": [1,2,3,4,5,6]
}

In [18]:
finalResult = MyGridSearch(df, param_dict)

Search Space Size:54


  0%|          | 0/54 [00:00<?, ?it/s]

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

[{'auc_mean': 0.6094999999999999, 'auc_std': 0.16195215342810357, 'auc': [[0.7, 0.5, 0.3, 0.49999999999999994, 0.7], [0.75, 0.7, 0.8, 0.7, 1.0], [0.6000000000000001, 0.65, 0.65, 0.7, 0.8], [0.4375, 0.5625, 0.4375, 0.875, 0.5625], [0.5, 0.4375, 0.4375, 0.5, 0.4375]], 'param': {'lr': 1e-07, 'epoch': 3}}, {'auc_mean': 0.5920000000000001, 'auc_std': 0.19147062437878035, 'auc': [[1.0, 1.0, 0.7, 0.49999999999999994, 0.4], [0.5, 0.6499999999999999, 0.5, 0.3, 0.5], [0.75, 0.25, 0.5, 0.75, 0.5], [0.75, 0.4375, 0.25, 0.6875, 0.75], [0.6875, 0.625, 0.625, 0.5, 0.6875]], 'param': {'lr': 1e-05, 'epoch': 5}}, {'auc_mean': 0.5795, 'auc_std': 0.15822768405054788, 'auc': [[0.15000000000000002, 0.6000000000000001, 0.5, 0.65, 0.55], [0.65, 0.65, 0.75, 0.3, 0.75], [0.7, 0.6, 0.75, 0.65, 0.8], [0.5625, 0.5, 0.625, 0.4375, 0.75], [0.5, 0.5625, 0.3125, 0.75, 0.4375]], 'param': {'lr': 5e-08, 'epoch': 4}}, {'auc_mean': 0.5765, 'auc_std': 0.14947073292119767, 'auc': [[0.7, 0.5, 0.55, 0.7, 0.6], [0.3, 0.85000000

## BERT Result

In [20]:
# BERT Result
finalResult

[{'auc_mean': 0.6094999999999999,
  'auc_std': 0.16195215342810357,
  'auc': [[0.7, 0.5, 0.3, 0.49999999999999994, 0.7],
   [0.75, 0.7, 0.8, 0.7, 1.0],
   [0.6000000000000001, 0.65, 0.65, 0.7, 0.8],
   [0.4375, 0.5625, 0.4375, 0.875, 0.5625],
   [0.5, 0.4375, 0.4375, 0.5, 0.4375]],
  'param': {'lr': 1e-07, 'epoch': 3}},
 {'auc_mean': 0.5920000000000001,
  'auc_std': 0.19147062437878035,
  'auc': [[1.0, 1.0, 0.7, 0.49999999999999994, 0.4],
   [0.5, 0.6499999999999999, 0.5, 0.3, 0.5],
   [0.75, 0.25, 0.5, 0.75, 0.5],
   [0.75, 0.4375, 0.25, 0.6875, 0.75],
   [0.6875, 0.625, 0.625, 0.5, 0.6875]],
  'param': {'lr': 1e-05, 'epoch': 5}},
 {'auc_mean': 0.5795,
  'auc_std': 0.15822768405054788,
  'auc': [[0.15000000000000002, 0.6000000000000001, 0.5, 0.65, 0.55],
   [0.65, 0.65, 0.75, 0.3, 0.75],
   [0.7, 0.6, 0.75, 0.65, 0.8],
   [0.5625, 0.5, 0.625, 0.4375, 0.75],
   [0.5, 0.5625, 0.3125, 0.75, 0.4375]],
  'param': {'lr': 5e-08, 'epoch': 4}},
 {'auc_mean': 0.5765,
  'auc_std': 0.149470732921

# Useless code, just for testing

In [21]:
device, _, _ = get_backend()
# model_name = "bert-base-uncased"
model_name = "/data/project/deberta-v3-base"

In [34]:
del model
del optimizer
# del metric
del batch
del lr_scheduler
del outputs
del loss
del logits
del predictions
del progress_bar
clear_device_cache()

In [22]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [24]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
# print(model)
optimizer = AdamW(model.parameters(), lr=1e-6)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
# )

model.to(device)

# from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        #lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
# metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
finalResult = []
finalLabel = []
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items() if k != "summarization"}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.sigmoid(logits)
    # print(predictions)
    finalResult += predictions.tolist()
    finalLabel += batch["labels"].tolist()
    # metric.add_batch(predictions=predictions, references=batch["labels"])
print(finalResult)
print(finalLabel)
print(roc_auc_score(finalLabel, finalResult))
# metric.compute()

[[0.597388505935669], [0.5704554319381714], [0.5689945220947266], [0.556196928024292], [0.5696896314620972], [0.5827513337135315], [0.5706762075424194], [0.5444836616516113], [0.5792657732963562], [0.5740975141525269], [0.5528358817100525]]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]
0.6428571428571428


In [30]:
roc_auc_score(finalLabel, finalResult)

0.7142857142857142